In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import numpy as np
import pandas as pd
from watttime import WattTimeHistorical, WattTimeForecast
import data.dataset as od
import evaluation.eval_framework as evu
import data.s3 as s3u
import math

import seaborn as sns
import matplotlib.pyplot as plt
import evaluation.battery as b

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

In [43]:
def make_test_moer_data():
    """Constant MOER at 1000.0 lbs/MWh except a single interval with 0.0"""
    start = pd.Timestamp("2023-12-13T01:45:00+00:00")
    end = pd.Timestamp("2023-12-13T11:40:00+00:00")
    moer_data = pd.DataFrame(columns=["point_time"], data=pd.date_range(start, end, freq="300s"))
    moer_data["value"] = 1000.0
    moer_data.loc[60, "value"] = 0.0
    return moer_data

def make_test_batteries():
    """Make 5 batteries which each have a fast charging part at a different point in the SoE"""
    batteries = []
    for i in range(1,4):
        charging_curve = pd.DataFrame(
            columns=["SoC", "kW"],
                data = [
                [0.0, 10.0],
                [i/5, 10.0],
                [i/5+0.01, 50.0],
                [i/5+0.20, 50.0],
                [i/5+0.21, 10.0],
                [1.0, 10],
            ]
        )
        battery = b.Battery(
            initial_soc = 0.0,
            charging_curve = charging_curve,
            capacity_kWh = 10
        )
        batteries.append(battery)
    return batteries

#battery.plot_charging_curve()


---